In [31]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

intents = json.loads(open('intents.json').read())

words = []
classes = []
documents = []
ignoreLetters = ['?', '!', '.', ',']

for intent in intents['intents']:
    for pattern in intent['Patterns']:
        wordList = nltk.word_tokenize(pattern)
        words.extend(wordList)
        documents.append((wordList, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [lemmatizer.lemmatize(word) for word in words if word not in ignoreLetters]
words = sorted(set(words))

classes = sorted(set(classes))

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

training = []
outputEmpty = [0] * len(classes)

for document in documents:
    bag = []
    wordPatterns = document[0]
    wordPatterns = [lemmatizer.lemmatize(word.lower()) for word in wordPatterns]
    for word in words:
        bag.append(1) if word in wordPatterns else bag.append(0)

    outputRow = list(outputEmpty)
    outputRow[classes.index(document[1])] = 1
    training.append(bag + outputRow)

random.shuffle(training)
training = np.array(training)

trainX = training[:, :len(words)]
trainY = training[:, len(words):]
print(documents)

model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(128, input_shape=(len(trainX[0]),), activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(64, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(len(trainY[0]), activation='softmax'))

sgd = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist=model.fit(trainX, trainY, epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5',hist)
print('Done')




[(['hello'], 'greetings'), (['hey'], 'greetings'), (['hi'], 'greetings'), (['good', 'day'], 'greetings'), (['Greetings'], 'greetings'), (['what', "'s", 'up', '?'], 'greetings'), (['how', 'is', 'it', 'going'], 'greetings'), (['I', 'am', 'leaving'], 'goodbye'), (['Goodbye'], 'goodbye'), (['bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Have', 'a', 'good', 'day'], 'goodbye'), (['Give', 'the', 'Smartwatch', 'Details'], 'Smartwatches_Details'), (['What', 'is', 'the', 'most', 'sold', 'Routers', 'Model', '3', 'in', 'india'], 'Router_model_3'), (['What', 'is', 'the', 'most', 'sold', 'Routers', 'Model', 'in', 'india'], 'Router_model_2'), (['What', 'is', 'the', 'total', 'number', 'of', 'routers', 'in', 'india', 'and', 'what', 'was', 'their', 'price'], 'total_routers'), (['What', 'is', 'the', 'most', 'sold', 'machines', 'in', 'India', 'last', 'three', 'years', 'and', 'in', 'the', 'last', '3', 'months'], 'most_sold'), (['What', 'is', 'the', 'most', 'sold', 'machines', 'in', 'India', '

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


6/6 [==============================] - 0s 4ms/step - loss: 2.7921 - accuracy: 0.0385
Epoch 2/200
6/6 [==============================] - 0s 2ms/step - loss: 2.8180 - accuracy: 0.1538
Epoch 3/200
6/6 [==============================] - 0s 2ms/step - loss: 2.5849 - accuracy: 0.1154
Epoch 4/200
6/6 [==============================] - 0s 2ms/step - loss: 2.5403 - accuracy: 0.2308
Epoch 5/200
6/6 [==============================] - 0s 2ms/step - loss: 2.4744 - accuracy: 0.4231
Epoch 6/200
6/6 [==============================] - 0s 2ms/step - loss: 2.4690 - accuracy: 0.2692
Epoch 7/200
6/6 [==============================] - 0s 2ms/step - loss: 2.3278 - accuracy: 0.5000
Epoch 8/200
6/6 [==============================] - 0s 2ms/step - loss: 2.1617 - accuracy: 0.3462
Epoch 9/200
6/6 [==============================] - 0s 2ms/step - loss: 2.1391 - accuracy: 0.3462
Epoch 10/200
6/6 [==============================] - 0s 2ms/step - loss: 1.9740 - accuracy: 0.5000
Epoch 11/200
6/6 [=======================

In [ ]:
import random
import json
import pickle
import numpy as np
import tensorflow as tf

import nltk
from nltk.stem import WordNetLemmatizer
from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words=pickle.load(open('words.pkl','rb'))
classes=pickle.load(open('classes.pkl','rb'))

model= load_model('chatbot_model.h5')

def clean_up_sentence (sentence):
  sentence_words=nltk.word_tokenize(sentence)
  sentence_words=[lemmatizer.lemmatize(word) for word in sentence_words]
  return sentence_words

def bag_of_words(sentence):
  sentence_words=clean_up_sentence(sentence)
  bag=[0]*len(words)
  for w in sentence_words:
    for i,word in enumerate(words):
        if words==w:
          bag[i]=1
  return np.array(bag)

def predict_class(sentence):
  bow=bag_of_words(sentence)
  res=model.predict(np.array([bow]))[0]
  ERROR_THRESH=0.25
  results=[[i,r] for i,r in enumerate(res) if r > ERROR_THRESH]

  results.sort(key=lambda x:x[1],reverse=True)
  return_list=[]
  for r in  results:
    return_list.append({'intent': classes[r[0]],'probability':str(r[1])})
  return return_list

def get_response (intents_list,intents_json):
  tag=intents_list[0]['intent']
  list_of_intents=intents_json['intents']
  for i in list_of_intents:
    if i['tag']==tag:
      result=random.choice(i['Responses'])
      break
  return result

print("Chatbot running")

while True:
  message=input("")
  ints=predict_class(message)
  res=get_response(ints,intents)
  print(res)






Chatbot running
hi
1/1 [==============================] - 0s 116ms/step
what can I do for you?
What is the most sold Routers Model in india
1/1 [==============================] - 0s 17ms/step
what can I do for you?
hello
1/1 [==============================] - 0s 17ms/step
Hello
What is the most sold Routers Model in india
1/1 [==============================] - 0s 19ms/step
what can I do for you?
What is the most sold Routers Model in india
1/1 [==============================] - 0s 19ms/step
what can I do for you?
bye
1/1 [==============================] - 0s 17ms/step
what can I do for you?
